In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
model = joblib.load('models\stock_predictor_MSFT.pkl')

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_29484\999767914.py:1: SyntaxWarning: invalid escape sequence '\s'
  model = joblib.load('models\stock_predictor_MSFT.pkl')


In [3]:
new_df = pd.read_pickle('NetGEX_AbsGEX_EPS(MSFT).pickle')

# Select numeric features and handle missing/infinite values
X_new = new_df.select_dtypes(include=[np.number]).copy()
X_new.replace([np.inf, -np.inf], np.nan, inplace=True)
X_new.fillna(X_new.median(), inplace=True)

In [16]:
import pickle
np.random.seed(42)

# ====================================
# 2. Data Loading
# ====================================
def load_pickle_data(file_path):
    """
    Attempt to load a DataFrame from a pickle file.
    
    Args:
        file_path (str): The path to the pickle file.
        
    Returns:
        data (DataFrame): The loaded pandas DataFrame or None if file is not found.
    """
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        print(f"Data successfully loaded from: {file_path}")
        return data
    except FileNotFoundError:
        print(f"File '{file_path}' not found. Check the file path and try again.")
        return None

In [17]:


# File path provided by the user
file_path = "NetGEX_AbsGEX_EPS(AAPL).pickle"
data = load_pickle_data(file_path)

if data is not None:
    print("Columns in the loaded data:")
    print(data.columns.tolist())


Data successfully loaded from: NetGEX_AbsGEX_EPS(AAPL).pickle
Columns in the loaded data:
['Spot_Open', 'Spot_Close', 'Spot_High', 'Spot_Low', 'PCT_EPS_1mo_Open', 'PCT_EPS_1mo_Close', 'PCT_EPS_1mo_High', 'PCT_EPS_1mo_Low', 'open_abs_gex', 'high_abs_gex', 'low_abs_gex', 'close_abs_gex', 'volume_abs_gex', 'open_net_gex', 'high_net_gex', 'low_net_gex', 'close_net_gex', 'volume_net_gex']


C:\Users\Admin\AppData\Local\Temp\ipykernel_29484\417150072.py:19: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)


In [18]:
data['date'] = pd.to_datetime(data.index)
data.sort_values('date', inplace=True)

In [20]:
from sklearn.preprocessing import MinMaxScaler
# --- Feature: Daily Return ---
# Calculate daily return using Spot_Close.
data['daily_return'] = data['Spot_Close'].pct_change().fillna(0)

# --- Target Variable ---
# Binary target: 1 if next day's Spot_Close is higher, else 0.
data['target'] = (data['Spot_Close'].shift(-1) > data['Spot_Close']).astype(int)
data = data.iloc[:-1]  # Drop last row due to shift

# --- Feature Scaling ---
scaler = MinMaxScaler()
data['Spot_Close_scaled'] = scaler.fit_transform(data[['Spot_Close']])
data['daily_return_scaled'] = scaler.fit_transform(data[['daily_return']])

print("\nData after preprocessing and feature engineering:")


Data after preprocessing and feature engineering:


In [21]:

# --- Step 2: Basic Cleanup ---
df = data.copy()
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

# Clip outliers in GEX-related columns to reduce noise
gex_columns = [col for col in df.columns if 'gex' in col.lower()]
df[gex_columns] = df[gex_columns].clip(lower=df[gex_columns].quantile(0.01),
                                       upper=df[gex_columns].quantile(0.99), axis=1)


C:\Users\Admin\AppData\Local\Temp\ipykernel_29484\3592349862.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_29484\3592349862.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [22]:
df['spot_return'] = df['Spot_Close'].pct_change().fillna(0)
df['spot_rolling_mean'] = df['Spot_Close'].rolling(window=5).mean().fillna(method='bfill')
df['spot_rolling_std'] = df['Spot_Close'].rolling(window=5).std().fillna(method='bfill')

# Add target label: Buy (1), Sell (-1), Hold (0)
threshold = 0.001  # 0.1% movement
df['future_return'] = df['Spot_Close'].pct_change(periods=5).shift(-5)

C:\Users\Admin\AppData\Local\Temp\ipykernel_29484\616468694.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['spot_rolling_mean'] = df['Spot_Close'].rolling(window=5).mean().fillna(method='bfill')
C:\Users\Admin\AppData\Local\Temp\ipykernel_29484\616468694.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['spot_rolling_std'] = df['Spot_Close'].rolling(window=5).std().fillna(method='bfill')


In [24]:
feature_cols = [col for col in df.columns if col not in ['future_return', 'target']]
X = df[feature_cols]

In [29]:
X.drop(columns=['date'], inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_29484\1148953874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['date'], inplace=True)


In [31]:
y_pred = model.predict(X)